<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="ffffff">
Resnet code

</font>
</h1>

pariya branch

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image, display
import sys
import pandas as pd
from keras.layers import Activation, Dense
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Sequential
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [2]:
get_ipython().system('pip install efficientnet')

In [3]:
#from tensorflow.keras.applications import ResNet50
#from tensorflow.keras.applications.resnet50 import preprocess_input

base_model = tf.keras.applications.ResNet50(weights="imagenet",
    input_shape=(100, 100, 3),
    include_top=False)

In [8]:
base_model.trainable = True

fine_tune_at = 140

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 50, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [9]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(12, activation='softmax'))

In [10]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 4, 4, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               8388864   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)               

In [11]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_ds = train_datagen.flow_from_directory(
    r'E:\projectDataset\archive\sl_train',
    class_mode='categorical',
    target_size=(100,100),
    shuffle=True,
    batch_size=32
)


test_datagen = ImageDataGenerator(rescale=1./255)
test_ds_ = test_datagen.flow_from_directory(
    r'E:\projectDataset\archive\sl_test',
    class_mode='categorical',
    target_size=(100,100),
    batch_size=32
)



val_datagen = ImageDataGenerator(rescale=1./255)
val_ds = val_datagen.flow_from_directory(
    r'E:\projectDataset\archive\sl_valid',
    class_mode='categorical',
    target_size=(100,100),
    shuffle=True,
    batch_size=32
)



Found 2907 images belonging to 12 classes.
Found 1166 images belonging to 12 classes.
Found 1420 images belonging to 12 classes.


In [12]:
model.compile(optimizer='adam', loss = "categorical_crossentropy" , metrics =['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=10)

history2 = model.fit(
    train_ds,
    batch_size=32,
    epochs=27,
    validation_data=val_ds,
    initial_epoch=22,
    callbacks=[early_stopping]
    )
    

Epoch 23/27
91/91 [==============================] - 252s 3s/step - loss: 3.4520 - accuracy: 0.1047 - val_loss: 2.7178 - val_accuracy: 0.0711
Epoch 24/27
91/91 [==============================] - 218s 2s/step - loss: 2.4598 - accuracy: 0.1121 - val_loss: 2.4902 - val_accuracy: 0.0704
Epoch 25/27
91/91 [==============================] - 185s 2s/step - loss: 2.4420 - accuracy: 0.1272 - val_loss: 2.5118 - val_accuracy: 0.0711
Epoch 26/27
91/91 [==============================] - 185s 2s/step - loss: 2.4330 - accuracy: 0.1216 - val_loss: 2.5263 - val_accuracy: 0.0697
Epoch 27/27
91/91 [==============================] - 203s 2s/step - loss: 2.4337 - accuracy: 0.1222 - val_loss: 2.5348 - val_accuracy: 0.0697


In [14]:
model.save('../models/6fruit_disease_classifier_resnet_FT.h5')

In [17]:
#test_loss, test_accuracy = model.evaluate(test_ds_, steps=len(test_ds_)/32)
#print('Accuracy on test dataset:', test_accuracy)

In [15]:
#import matplotlib.pyplot as plt

#plt.semilogx( history2.history["loss"])

In [16]:
#plt.semilogx( history2.history["accuracy"])